In [12]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)

"""
Simple Moving Average
https://school.stockcharts.com/doku.php?id=technical_indicators:moving_averages
"""
def sma(data,period,column_name):
    # Turn SMA(Close_Values) Col to SMA
    data['SMA_'+ str(period)] = data[column_name].rolling(window=period).mean()
    return data


"""
Exponential Moving Average
https://school.stockcharts.com/doku.php?id=technical_indicators:moving_averages
"""
def ema(data,period,column_name):
    # ema formula
    data['EMA_'+ str(period)] = data[column_name].ewm(ignore_na=False,min_periods=period, span=period,adjust=True).mean()
    return data

"""
Moving Average Convergence/Divergence Oscillator (MACD)
Source: http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:moving_average_convergence_divergence_macd
"""
def macd(data,period_short,period_long,period_signal,column_name):
    # columns to drop
    cols2drop=[]
    
    # EMA short
    data['EMA_'+ str(period_short)] = data[column_name].ewm(ignore_na=False,min_periods=period_short, span=period_short,adjust=True).mean()
    ema_short=data['EMA_'+ str(period_short)]
    cols2drop.append('EMA_'+ str(period_short))
    
    # EMA long
    data['EMA_'+ str(period_long)] = data[column_name].ewm(ignore_na=False,min_periods=period_long, span=period_long,adjust=True).mean()
    ema_long=data['EMA_'+ str(period_long)]
    cols2drop.append('EMA_'+ str(period_long))

    # MACD Formula
    data_macd=ema_short - ema_long
    data['MACD_'+ str(period_short)+'_'+ str(period_long)] = data_macd
    data['MACD_signal_'+ str(period_signal)] = data_macd.ewm(ignore_na=False,min_periods=period_signal, span=period_signal,adjust=True).mean()
    
    # Drop Cols
    data.drop(cols2drop,axis=1, inplace=True)
    
    return data
"""
    Accumulation Distribution 
Source: http://stockcharts.com/school/doku.php?st=accumulation+distribution&id=chart_school:technical_indicators:accumulation_distribution_line
Params: 
    data: pandas DataFrame
    trend_periods: the over which to calculate AD
    open_col: the name of the OPEN values column
	high_col: the name of the HIGH values column
	low_col: the name of the LOW values column
	close_col: the name of the CLOSE values column
	vol_col: the name of the VOL values column
    
Returns:
    copy of 'data' DataFrame with 'acc_dist' and 'acc_dist_ema[trend_periods]' columns added
"""
    def acc_dist():


# Creating data array
b=np.arange(2,100,2) 
c=np.arange(100,2,-2)
d=[b,c]
d_array=np.resize(d,(1,(len(b)+len(c))))
d_array=d_array[0]

df = pd.DataFrame({'CLOSE': d_array})
sma_25=sma(df,25,'CLOSE')
ema_25=ema(df,25,'CLOSE')
macd_5_8=macd(df,26,12,9,'CLOSE')

df.head(80)

,CLOSE,SMA_25,EMA_25,MACD_26_12,MACD_signal_9
0,2,NaN,NaN,NaN,NaN
1,4,NaN,NaN,NaN,NaN
2,6,NaN,NaN,NaN,NaN
3,8,NaN,NaN,NaN,NaN
4,10,NaN,NaN,NaN,NaN
5,12,NaN,NaN,NaN,NaN
6,14,NaN,NaN,NaN,NaN
7,16,NaN,NaN,NaN,NaN
8,18,NaN,NaN,NaN,NaN
9,20,NaN,NaN,NaN,NaN
